# Mount Google Drive and import packages


In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Random seed 고정

In [2]:
import numpy as np
import random
import torch
import matplotlib.pyplot as plt

def set_global_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = False
    cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    cudnn.allow_tf32 = False
    torch.use_deterministic_algorithms(True, warn_only=True)

set_global_seed(42)

Data를 Google drvie -> Colab local로 변경

In [3]:
# 드라이브의 zip 파일을 Colab 로컬 저장소로 복사
!cp /content/drive/MyDrive/DCC2025/Data/Training/01.원천데이터/TS_KS.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Training/02.라벨링데이터/TL_KS_BBOX.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Validation/01.원천데이터/VS_KS.zip /content
!cp /content/drive/MyDrive/DCC2025/Data/Validation/02.라벨링데이터/VL_KS_BBOX.zip /content/

# 압축 해제
!unzip /content/TS_KS.zip -d /content/TS_KS
!unzip /content/TL_KS_BBOX.zip -d /content/TL_KS_BBOX
!unzip /content/VS_KS.zip -d /content/VS_KS
!unzip /content/VL_KS_BBOX.zip -d /content/VL_KS_BBOX

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053730_14.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221205052944_13.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_53.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221205052944_48.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_6.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_58.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_55.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_59.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_65.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_48.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221205052944_3.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_68.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_69.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221205052944_45.json  
  inflating: /content/TL_KS_BBOX/K3_CHN_20221125053731_6

In [4]:
!pip install ultralytics==8.3.183
!pip show ultralytics

import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:00:00
Name: ultralytics
Version: 8.3.183
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: https://ultralytics.com
Author: 
Author-email: Glenn Jocher <glenn.jocher@ultralytics.com>, Jing Qiu <jing.qiu@ultralytics.com>
License: AGPL-3.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, torch, torchvision, tqdm, ultralytics-thop
Required-by: 
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Check GPU

In [5]:
# CUDA, GPU 사용 가능 확인
import torch

print("CUDA available:", torch.cuda.is_available()) # True
print("Device count:", torch.cuda.device_count()) # 1
print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else None) # 0
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None) # Tesla T4, NVIDIA A100-SXM4-40GB

!nvidia-smi

CUDA available: True
Device count: 1
Current device: 0
Device name: NVIDIA L4
Tue Oct 14 13:55:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   50C    P8             13W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                   

# Make dataset for YOLO

In [6]:
import os, json, shutil, random
from pathlib import Path
from typing import List, Dict, Any, Tuple

For validation

In [7]:
# ---------- PATH ----------
BASE = Path("/content")
IMG_DIR = BASE / "TS_KS"
ANN_DIR = BASE / "TL_KS_BBOX"
OUT_DIR = BASE / "Mission_1"
LOG_PATH = OUT_DIR / "M1_convert_data.txt"

num_imgs = len(list(IMG_DIR.glob("*.jpg")))
num_anns = len(list(ANN_DIR.glob("*.json")))
print(f"Train 이미지 개수: {num_imgs}")
print(f"Train 어노테이션 개수: {num_anns}")

# ---------- SETTINGS ----------
RATIOS = (1.0, 0.0)    # train
SEED = 42
CLASS_NAME = "chimney"

Train 이미지 개수: 8052
Train 어노테이션 개수: 8052


In [8]:
# ---------- OUTPUT DIRS ----------
OUT_IMG = {k: OUT_DIR / f"images/{k}" for k in ["train", "val"]}
OUT_LAB = {k: OUT_DIR / f"labels/{k}" for k in ["train", "val"]}

Ground truth json파일을 YOLO Model 학습을 위한 형태로 변형

In [9]:
# ---------- Functions ----------
def yolo_box(x: int, y: int, w: int, h: int, W: int, H: int) -> Tuple[float, float, float, float]:
  # rect -> YOLO (cx, cy, w, h) normalized
    cx = (x + w / 2) / W
    cy = (y + h / 2) / H
    nw = w / W
    nh = h / H
    return cx, cy, nw, nh

In [10]:
def collect_records() -> List[Dict[str, Any]]:
    # 각 JSON에서 유효한 박스들을 수집해 records 반환.
    records = []
    json_files = sorted(ANN_DIR.glob("*.json"))
    if not json_files:
        raise SystemExit(f"No JSON files found in: {ANN_DIR}")

    OUT_DIR.mkdir(parents=True, exist_ok=True)
    with LOG_PATH.open("w", encoding="utf-8") as log:
        for jp in json_files:
            try:
                data = json.loads(jp.read_text(encoding="utf-8"))
                key = next(iter(data))
                item = data[key]
            except Exception as e:
                log.write(f"[ERR_JSON] {jp.name}: {e}\n")
                continue

            img_name = item.get("filename")
            if not img_name:
                log.write(f"[MISS_FILENAME] {jp.name}\n")
                continue

            img_path = IMG_DIR / img_name
            if not img_path.exists():
                log.write(f"[MISS_IMG] {img_name}\n")
                continue

            fa = item.get("file_attributes", {}) or {}
            try:
                H = int(fa.get("img_height"))
                W = int(fa.get("img_width"))
            except Exception as e:
                log.write(f"[MISS_SIZE] {img_name}: {e}\n")
                continue

            good_boxes = []
            regions = item.get("regions", []) or []
            for r in regions:
                sh = r.get("shape_attributes", {}) or {}
                if sh.get("name") != "rect":
                    continue
                try:
                    x = int(sh["x"]); y = int(sh["y"])
                    w = int(sh["width"]); h = int(sh["height"])
                except Exception as e:
                    log.write(f"[BAD_SHAPE] {img_name}: {e}\n")
                    continue

                if w <= 0 or h <= 0 or x < 0 or y < 0 or x + w > W or y + h > H:
                    log.write(f"[BAD_BOX] {img_name} | {x,y,w,h} in {W,H}\n")
                    continue

                good_boxes.append((x, y, w, h))

            if not good_boxes:
                log.write(f"[NO_BOX] {img_name}\n")
                continue

            records.append({"img": img_path, "size": (W, H), "boxes": good_boxes})

    if not records:
        raise SystemExit("No valid annotations found.")
    return records

def split_records(records: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    # seed를 고정시킨 뒤 분할
    random.seed(SEED)
    random.shuffle(records)
    n = len(records)
    n_train = int(n * RATIOS[0])
    n_val = int(n * RATIOS[1])
    split = {
        "train": records[:n_train],
        "val": records[n_train:n_train + n_val],
        }
    return split

In [11]:
def write_split(name: str, split_set: List[Dict[str, Any]]):
    # Yolo 용 데이터 생성
    img_out, lab_out = OUT_IMG[name], OUT_LAB[name]
    img_out.mkdir(parents=True, exist_ok=True)
    lab_out.mkdir(parents=True, exist_ok=True)

    for rec in split_set:
        src_img = rec["img"]
        dst_img = img_out / src_img.name

        if dst_img.exists():
            continue
        shutil.copy2(src_img, dst_img)

        W, H = rec["size"]
        lines = []
        for (x, y, w, h) in rec["boxes"]:
            cx, cy, nw, nh = yolo_box(x, y, w, h, W, H)
            lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")  # single class

        (lab_out / f"{src_img.stem}.txt").write_text("\n".join(lines), encoding="utf-8")

def write_yaml():
    yaml_text = f"""# auto-generated
      path: {OUT_DIR.as_posix()}
      train: images/train
      val: images/val
      test: images/val

      nc: 1
      names: ['{CLASS_NAME}']
      """
    (OUT_DIR / "data.yaml").write_text(yaml_text, encoding="utf-8")

In [12]:
records = collect_records()
split = split_records(records)
for k in ["train", "val"]:
    write_split(k, split[k])
write_yaml()

print(f"Done. Total: {len(records)} | "
      f"Train: {len(split['train'])}")

Done. Total: 8050 | Train: 8050


For traning

In [13]:
# ---------- PATH ----------
BASE = Path("/content")
IMG_DIR = BASE / "VS_KS"
ANN_DIR = BASE / "VL_KS_BBOX"
OUT_DIR = BASE / "Mission_1"
LOG_PATH = OUT_DIR / "M1_convert_data.txt"

num_imgs = len(list(IMG_DIR.glob("*.jpg")))
num_anns = len(list(ANN_DIR.glob("*.json")))
print(f"Val 이미지 개수: {num_imgs}")
print(f"Val 어노테이션 개수: {num_anns}")

# ---------- SETTINGS ----------
RATIOS = (0.0, 1.0)    # train

Val 이미지 개수: 1006
Val 어노테이션 개수: 1006


In [14]:
records = collect_records()
split = split_records(records)
for k in ["train", "val"]:
    write_split(k, split[k])
write_yaml()

print(f"Done. Total: {len(records)} | "
      f"Val: {len(split['val'])}")

Done. Total: 1006 | Val: 1006


# Train model

모델 학습 파라미터 설정 및 경로 설정

In [ ]:
# ============ CONFIG ============
DATA_YAML = BASE / "Mission_1/data.yaml"
DRIVE = Path("/content/drive/MyDrive/run")
PROJECT = DRIVE / "result"
RUN_NAME = "m1_final5"
DEVICE = 0
EPOCHS = 100
BATCH = 8
IMG_SIZE = 512
DO_TRAIN = True
# =================================

Data augmentation을 적용한 모델 학습 및 학습 결과 분석

In [ ]:
def train_yolov11():
    model = YOLO("yolo11m.pt")
    results = model.train(
        data=str(DATA_YAML),
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMG_SIZE,
        device=DEVICE,
        cache=True,
        project=str(PROJECT),
        name=RUN_NAME,
        seed=42,
        pretrained=True,
        verbose=True,
        crop_fraction = 0.4,
        cutmix=0.1,
        fliplr=0.2,
        save=True,
    )
    return Path(results.save_dir)

def find_results_csv(run_dir: Path) -> Path:
    csv = run_dir / "results.csv"
    if not csv.exists():
        raise FileNotFoundError(f"results.csv not found under {run_dir}")
    return csv

def load_results_df(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path, skipinitialspace=True, engine="python")
    df.columns = [c.strip() for c in df.columns]

    if "epoch" not in df.columns:
        cand = [c for c in df.columns if c.lower().endswith("epoch")]
        if cand:
            df = df.rename(columns={cand[0]: "epoch"})
    if "epoch" not in df.columns:
        df = df.rename(columns={df.columns[0]: "epoch"})

    for c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="ignore")
    df["epoch"] = pd.to_numeric(df["epoch"], errors="coerce").astype("Int64")
    df = df.dropna(subset=["epoch"]).sort_values("epoch").reset_index(drop=True)
    return df

def plot_losses(df: pd.DataFrame, out_path: Path):
    def cols_exist(cands): return [c for c in cands if c in df.columns]

    train_loss_cols = cols_exist(["train/box_loss","train/cls_loss","train/dfl_loss"])
    val_loss_cols   = cols_exist(["val/box_loss","val/cls_loss","val/dfl_loss"])

    if train_loss_cols:
        df["train/total_loss"] = df[train_loss_cols].apply(pd.to_numeric, errors="coerce").sum(axis=1)
    if val_loss_cols:
        df["val/total_loss"]   = df[val_loss_cols].apply(pd.to_numeric, errors="coerce").sum(axis=1)

    plt.figure(figsize=(8,5))
    if "train/total_loss" in df.columns and "val/total_loss" in df.columns:
        plt.plot(df["epoch"], df["train/total_loss"], label="train/total_loss")
        plt.plot(df["epoch"], df["val/total_loss"],   label="val/total_loss")
    else:
        for c in train_loss_cols:
            plt.plot(df["epoch"], pd.to_numeric(df[c], errors="coerce"), label=c)
        for c in val_loss_cols:
            plt.plot(df["epoch"], pd.to_numeric(df[c], errors="coerce"), label=c)

    plt.xlabel("epoch"); plt.ylabel("loss"); plt.title("Train/Val Loss")
    plt.legend(); plt.grid(True, alpha=0.3)
    plt.tight_layout()
    out_path.mkdir(exist_ok=True, parents=True)
    plt.savefig(out_path / "loss_curves.png", dpi=150)
    plt.close()

def plot_accuracy(df: pd.DataFrame, out_path: Path):
    # mAP/PR/RC 컬럼 존재 여부 확인 후 그리기
    m50   = "metrics/mAP50(B)"
    m5095 = "metrics/mAP50-95(B)"
    prec  = "metrics/precision(B)"
    reca  = "metrics/recall(B)"

    plt.figure(figsize=(8,5))
    if m50 in df.columns:
        plt.plot(df["epoch"], pd.to_numeric(df[m50], errors="coerce"), label="mAP@0.50")
    elif m5095 in df.columns:
        plt.plot(df["epoch"], pd.to_numeric(df[m5095], errors="coerce"), label="mAP@0.50:0.95")
    if prec in df.columns:
        plt.plot(df["epoch"], pd.to_numeric(df[prec], errors="coerce"), label="precision")
    if reca in df.columns:
        plt.plot(df["epoch"], pd.to_numeric(df[reca], errors="coerce"), label="recall")

    plt.xlabel("epoch"); plt.ylabel("score"); plt.title("Validation Metrics")
    plt.legend(); plt.grid(True, alpha=0.3)
    plt.tight_layout()
    out_path.mkdir(exist_ok=True, parents=True)
    plt.savefig(out_path / "accuracy_curves.png", dpi=150)
    plt.close()

In [ ]:
    run_dir = PROJECT / RUN_NAME
    if DO_TRAIN:
        run_dir = train_yolov11()

    results_csv = find_results_csv(run_dir)
    print("results.csv:", results_csv)
    df = load_results_df(results_csv)

    # 그래프 저장 디렉토리
    plots_dir = run_dir / "plots_custom"
    plots_dir.mkdir(exist_ok=True)

    plot_losses(df, plots_dir)
    plot_accuracy(df, plots_dir)

    print("Saved plots:")
    print(plots_dir / "loss_curves.png")
    print(plots_dir / "accuracy_curves.png")

New https://pypi.org/project/ultralytics/8.3.213 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'crop_fraction' is deprecated and will be removed in in the future.
Ultralytics 8.3.183 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.1, data=/content/Mission_1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.2, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0,

train: Scanning /content/Mission_1/labels/train.cache... 8050 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8050/8050 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (5.9GB RAM): 100%|██████████| 8050/8050 [00:03<00:00, 2451.10it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

    100/100         3G     0.5418      0.287      0.854          3        512: 100%|██████████| 1007/1007 [02:00<00:00,  8.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:06<00:00,  9.30it/s]

                   all       1006       1322      0.986      0.988      0.994      0.884



100 epochs completed in 3.532 hours.
Optimizer stripped from /content/drive/MyDrive/run/result/m1_final5/weights/last.pt, 40.5MB
Optimizer stripped from /content/drive/MyDrive/run/result/m1_final5/weights/best.pt, 40.5MB

Validating /content/drive/MyDrive/run/result/m1_final5/weights/best.pt...
Ultralytics 8.3.183 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:06<00:00,  9.15it/s]


                   all       1006       1322      0.986      0.988      0.994      0.884
Speed: 0.1ms preprocess, 3.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/run/result/m1_final5
results.csv: /content/drive/MyDrive/run/result/m1_final5/results.csv
Saved plots:
/content/drive/MyDrive/run/result/m1_final5/plots_custom/loss_curves.png
/content/drive/MyDrive/run/result/m1_final5/plots_custom/accuracy_curves.png


# Validation Model

In [16]:
model = YOLO(str("/content/drive/MyDrive/run/result/m1_final5/weights/best.pt"))

metrics = model.val(
    data=str(DATA_YAML),
    split="test",      # ← 반드시 test!
    imgsz=IMG_SIZE,
    device=DEVICE,
    save_json=False,   # COCO json 내보내기 필요시 True
    batch=16,          # GPU 여유에 맞게 조절
    iou=0.5,           # IoU threshold for mAP calc (기본 0.7)
    verbose=True,
)

print("\n=== Test metrics ===")
try:
    print(f"mAP@0.50:      {metrics.box.map50:.4f}")
    print(f"mAP@0.50:0.95: {metrics.box.map:.4f}")
    print(f"Precision:     {metrics.box.mp:.4f}")
    print(f"Recall:        {metrics.box.mr:.4f}")
except Exception:
    print(metrics)

Ultralytics 8.3.183 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1825.6±1080.8 MB/s, size: 86.3 KB)



val: Scanning /content/Mission_1/labels/val... 1006 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1006/1006 [00:00<00:00, 1524.33it/s]

val: New cache created: /content/Mission_1/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:13<00:00,  4.58it/s]


                   all       1006       1322      0.986      0.988      0.994      0.885
Speed: 0.1ms preprocess, 8.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val

=== Test metrics ===
mAP@0.50:      0.9941
mAP@0.50:0.95: 0.8855
Precision:     0.9864
Recall:        0.9882
